# Week 22: System Design for Quantitative Trading - Theory

## 🎯 Learning Objectives

By the end of this week, you will understand:
- **Trading System Architecture**: Components, patterns, and principles
- **Low-Latency Design**: Techniques for microsecond-level performance
- **Data Infrastructure**: Time-series databases, streaming, and storage
- **Distributed Systems**: Scalability, fault tolerance, and consistency
- **Production Deployment**: Monitoring, alerting, and incident response

---

## Why System Design for Quant?

> "In quantitative trading, system design is as important as the alpha itself."

- Alpha decay is fast → need rapid iteration
- Market changes → need adaptable systems
- Competition is fierce → latency matters
- Capital at risk → reliability is critical

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Any
from enum import Enum, auto
from abc import ABC, abstractmethod
import time
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
print("✅ Libraries loaded!")
print("📚 Week 22: System Design Theory")

✅ Libraries loaded!
📚 Week 22: System Design Theory


---

## Part 1: Trading System Architecture

### 1.1 Core Components

```
┌─────────────────────────────────────────────────────────────────┐
│                    TRADING SYSTEM ARCHITECTURE                   │
├─────────────────────────────────────────────────────────────────┤
│                                                                  │
│  ┌─────────────┐    ┌─────────────┐    ┌─────────────┐          │
│  │ DATA LAYER  │───▶│ STRATEGY    │───▶│ EXECUTION   │          │
│  │             │    │ LAYER       │    │ LAYER       │          │
│  └─────────────┘    └─────────────┘    └─────────────┘          │
│        │                  │                  │                   │
│        ▼                  ▼                  ▼                   │
│  ┌─────────────────────────────────────────────────────────┐    │
│  │              INFRASTRUCTURE LAYER                        │    │
│  │  (Messaging, Databases, Monitoring, Risk)               │    │
│  └─────────────────────────────────────────────────────────┘    │
│                                                                  │
└─────────────────────────────────────────────────────────────────┘
```

### Component Breakdown

| Component | Responsibility | Latency Requirement |
|-----------|---------------|---------------------|
| Market Data Handler | Ingest, normalize, distribute | < 100μs |
| Strategy Engine | Signal generation | < 1ms |
| Risk Manager | Pre/post-trade checks | < 50μs |
| Order Management | Order lifecycle | < 100μs |
| Execution Engine | Smart routing | < 500μs |
| Portfolio Manager | P&L, positions | < 10ms |

### 1.2 Architecture Patterns

#### Event-Driven Architecture (EDA)

```
MARKET_DATA → SIGNAL → ORDER → FILL → PORTFOLIO_UPDATE
     │          │        │       │           │
     └──────────┴────────┴───────┴───────────┘
                     EVENT BUS
```

**Benefits:**
- Loose coupling between components
- Easy to add new consumers
- Natural audit trail
- Replay capability for backtesting

**Drawbacks:**
- Added latency from serialization
- Complexity in debugging
- Eventual consistency challenges

#### Shared Memory Architecture

Used by HFT firms for ultra-low latency:

```
┌─────────────────────────────────────────────┐
│            SHARED MEMORY REGION             │
│  ┌─────────┬─────────┬─────────┬────────┐  │
│  │ Order   │ Market  │ Position│ Risk   │  │
│  │ Book    │ Data    │ Cache   │ Limits │  │
│  └─────────┴─────────┴─────────┴────────┘  │
└─────────────────────────────────────────────┘
         │        │        │        │
    Strategy  Execution  Risk   Portfolio
```

**Key Techniques:**
- Memory-mapped files
- Lock-free data structures
- CPU pinning and NUMA awareness

In [2]:
# Event-Driven Architecture Example

class EventType(Enum):
    MARKET_DATA = auto()
    SIGNAL = auto()
    ORDER = auto()
    FILL = auto()
    PORTFOLIO_UPDATE = auto()

@dataclass
class Event:
    """Base event class"""
    event_type: EventType
    timestamp: int  # nanoseconds
    data: Dict[str, Any] = field(default_factory=dict)

class EventBus:
    """Central event dispatcher"""
    
    def __init__(self):
        self.subscribers: Dict[EventType, List] = {}
        self.event_count = 0
    
    def subscribe(self, event_type: EventType, handler):
        if event_type not in self.subscribers:
            self.subscribers[event_type] = []
        self.subscribers[event_type].append(handler)
    
    def publish(self, event: Event):
        self.event_count += 1
        for handler in self.subscribers.get(event.event_type, []):
            handler(event)

# Demo
bus = EventBus()

def log_handler(event):
    print(f"📨 Received {event.event_type.name}: {event.data}")

bus.subscribe(EventType.MARKET_DATA, log_handler)
bus.subscribe(EventType.SIGNAL, log_handler)

# Simulate event flow
bus.publish(Event(EventType.MARKET_DATA, time.time_ns(), {'symbol': 'AAPL', 'price': 150.25}))
bus.publish(Event(EventType.SIGNAL, time.time_ns(), {'symbol': 'AAPL', 'direction': 'LONG'}))

print(f"\n✅ Total events processed: {bus.event_count}")

📨 Received MARKET_DATA: {'symbol': 'AAPL', 'price': 150.25}
📨 Received SIGNAL: {'symbol': 'AAPL', 'direction': 'LONG'}

✅ Total events processed: 2


---

## Part 2: Low-Latency Design Principles

### 2.1 The Latency Ladder

| Operation | Latency | Context |
|-----------|---------|----------|
| L1 cache access | 1 ns | Best case |
| L2 cache access | 4 ns | Still fast |
| L3 cache access | 40 ns | Shared across cores |
| Main memory | 100 ns | RAM access |
| SSD read | 100 μs | 1000x slower than RAM |
| Network (same DC) | 500 μs | Add serialization |
| Network (cross DC) | 50 ms | Avoid for hot path |

### 2.2 Key Optimization Techniques

1. **Minimize Allocations**
   - Object pooling
   - Pre-allocated buffers
   - Avoid garbage collection pauses

2. **Reduce Serialization**
   - Binary protocols (FlatBuffers, Cap'n Proto)
   - Shared memory for intra-process

3. **Cache Optimization**
   - Data locality (keep related data together)
   - Cache-line alignment
   - Avoid false sharing

4. **Lock-Free Algorithms**
   - Compare-and-swap (CAS)
   - SPSC queues (single producer/consumer)
   - Disruptor pattern

In [3]:
# Latency Measurement Utilities

import statistics

def measure_latency(func, iterations=10000):
    """Measure function latency in microseconds"""
    latencies = []
    
    for _ in range(iterations):
        start = time.perf_counter_ns()
        func()
        end = time.perf_counter_ns()
        latencies.append((end - start) / 1000)  # Convert to μs
    
    return {
        'mean': statistics.mean(latencies),
        'median': statistics.median(latencies),
        'p99': np.percentile(latencies, 99),
        'p999': np.percentile(latencies, 99.9),
        'max': max(latencies)
    }

# Compare dict vs pre-allocated array access
test_dict = {f'key_{i}': i for i in range(1000)}
test_array = np.arange(1000)

def dict_access():
    return test_dict['key_500']

def array_access():
    return test_array[500]

print("Latency Comparison (μs)")
print("="*50)

dict_stats = measure_latency(dict_access)
array_stats = measure_latency(array_access)

print(f"{'Metric':<10} {'Dict':<12} {'Array':<12} {'Ratio'}")
print("-"*50)
for metric in ['mean', 'median', 'p99']:
    ratio = dict_stats[metric] / array_stats[metric] if array_stats[metric] > 0 else float('inf')
    print(f"{metric:<10} {dict_stats[metric]:<12.3f} {array_stats[metric]:<12.3f} {ratio:.1f}x")

Latency Comparison (μs)
Metric     Dict         Array        Ratio
--------------------------------------------------
mean       0.104        0.123        0.8x
median     0.084        0.125        0.7x
p99        0.250        0.167        1.5x


### 2.3 Object Pooling Pattern

Avoid allocation overhead by reusing objects:

```
WITHOUT POOLING:                WITH POOLING:
─────────────────               ──────────────
request → allocate → use       request → get from pool → use
         → GC (pause!)                  → return to pool
```

In [4]:
from collections import deque

@dataclass
class Order:
    """Order object that can be pooled"""
    symbol: str = ""
    side: str = ""
    quantity: int = 0
    price: float = 0.0
    timestamp: int = 0
    
    def reset(self):
        """Reset for reuse"""
        self.symbol = ""
        self.side = ""
        self.quantity = 0
        self.price = 0.0
        self.timestamp = 0

class ObjectPool:
    """Generic object pool for reducing allocations"""
    
    def __init__(self, factory, initial_size=100):
        self.factory = factory
        self.pool = deque(factory() for _ in range(initial_size))
        self.allocations = 0
        self.reuses = 0
    
    def acquire(self):
        """Get an object from pool"""
        if self.pool:
            self.reuses += 1
            return self.pool.popleft()
        self.allocations += 1
        return self.factory()
    
    def release(self, obj):
        """Return object to pool"""
        if hasattr(obj, 'reset'):
            obj.reset()
        self.pool.append(obj)
    
    def stats(self):
        return {
            'pool_size': len(self.pool),
            'allocations': self.allocations,
            'reuses': self.reuses,
            'reuse_rate': self.reuses / (self.allocations + self.reuses) if (self.allocations + self.reuses) > 0 else 0
        }

# Demo
order_pool = ObjectPool(Order, initial_size=50)

# Simulate order flow
for i in range(1000):
    order = order_pool.acquire()
    order.symbol = "AAPL"
    order.quantity = 100
    # ... process order ...
    order_pool.release(order)

stats = order_pool.stats()
print("Object Pool Statistics")
print("="*40)
for k, v in stats.items():
    print(f"{k}: {v:.2%}" if 'rate' in k else f"{k}: {v}")

Object Pool Statistics
pool_size: 50
allocations: 0
reuses: 1000
reuse_rate: 100.00%


---

## Part 3: Data Infrastructure

### 3.1 Time-Series Databases

| Database | Use Case | Write Speed | Query Speed |
|----------|----------|-------------|-------------|
| InfluxDB | Metrics, monitoring | Fast | Good |
| TimescaleDB | OHLCV, trades | Good | Excellent (SQL) |
| QuestDB | Tick data | Excellent | Excellent |
| Arctic | Research, backtesting | Good | Good |
| kdb+/q | HFT, real-time | Excellent | Excellent |

### 3.2 Data Hierarchy

```
HOT DATA (Real-time)           WARM DATA (Recent)         COLD DATA (Historical)
────────────────────           ─────────────────          ─────────────────────
- In-memory (Redis)            - SSD (Time-series DB)     - Object Storage (S3)
- Shared memory                - Compressed columns       - Parquet files
- Sub-millisecond access       - Second-level queries     - Batch processing
```

### 3.3 Message Queues for Trading

| Queue | Latency | Throughput | Durability | Use Case |
|-------|---------|------------|------------|----------|
| Aeron | 1-10 μs | 100M/s | Optional | Ultra-low latency |
| LMAX Disruptor | 10 μs | 10M/s | No | In-process |
| Kafka | 1-10 ms | 1M/s | Yes | Event sourcing |
| Redis Streams | 100 μs | 100K/s | Yes | General purpose |

In [5]:
# Ring Buffer (Disruptor-like) Implementation

class RingBuffer:
    """
    Lock-free ring buffer for single producer/consumer.
    Used in low-latency trading systems.
    """
    
    def __init__(self, size: int):
        # Size must be power of 2 for efficient modulo
        self.size = 1 << (size - 1).bit_length()  # Round up to power of 2
        self.mask = self.size - 1
        self.buffer = [None] * self.size
        self.write_pos = 0
        self.read_pos = 0
    
    def write(self, item) -> bool:
        """Write item to buffer. Returns False if full."""
        next_write = (self.write_pos + 1) & self.mask
        if next_write == self.read_pos:
            return False  # Buffer full
        self.buffer[self.write_pos] = item
        self.write_pos = next_write
        return True
    
    def read(self):
        """Read item from buffer. Returns None if empty."""
        if self.read_pos == self.write_pos:
            return None  # Buffer empty
        item = self.buffer[self.read_pos]
        self.read_pos = (self.read_pos + 1) & self.mask
        return item
    
    def __len__(self):
        return (self.write_pos - self.read_pos) & self.mask

# Demo
rb = RingBuffer(1024)

# Simulate market data flow
for i in range(100):
    rb.write({'seq': i, 'price': 100 + np.random.randn()})

print(f"Ring buffer size: {rb.size}")
print(f"Items in buffer: {len(rb)}")
print(f"\nFirst 5 items:")
for _ in range(5):
    print(f"  {rb.read()}")

Ring buffer size: 1024
Items in buffer: 100

First 5 items:
  {'seq': 0, 'price': 100.49671415301123}
  {'seq': 1, 'price': 99.86173569882881}
  {'seq': 2, 'price': 100.6476885381007}
  {'seq': 3, 'price': 101.52302985640803}
  {'seq': 4, 'price': 99.76584662527667}


---

## Part 4: Distributed Systems Concepts

### 4.1 CAP Theorem

**Choose 2 of 3:**
- **Consistency**: All nodes see the same data
- **Availability**: Every request gets a response
- **Partition Tolerance**: System works despite network failures

```
Trading System Trade-offs:
─────────────────────────────────────────────────────────────
Order Book → CP (Consistency + Partition Tolerance)
             Can't have stale order book state!

Market Data Cache → AP (Availability + Partition Tolerance)
             Stale is OK, unavailable is not

P&L Calculation → CP
             Must be accurate for risk management
```

### 4.2 Consensus Algorithms

| Algorithm | Use Case | Latency | Complexity |
|-----------|----------|---------|------------|
| Raft | Leader election | Medium | Low |
| Paxos | Distributed state | Medium | High |
| ZAB (ZooKeeper) | Configuration | Medium | Medium |

### 4.3 Replication Strategies

```
SYNCHRONOUS REPLICATION:        ASYNCHRONOUS REPLICATION:
────────────────────────        ──────────────────────────
Client → Primary → Replica     Client → Primary
              ↓    ↓                         ↓
          Wait for ACK              ACK (async to replica)
              ↓
           ACK to client

Pros: Strong consistency        Pros: Lower latency
Cons: Higher latency            Cons: Potential data loss
```

In [6]:
# Simple Raft-like Leader Election Simulation

import random

class Node:
    def __init__(self, node_id: int, total_nodes: int):
        self.node_id = node_id
        self.total_nodes = total_nodes
        self.state = "FOLLOWER"  # FOLLOWER, CANDIDATE, LEADER
        self.term = 0
        self.voted_for = None
        self.votes_received = 0
    
    def start_election(self):
        """Become candidate and request votes"""
        self.state = "CANDIDATE"
        self.term += 1
        self.voted_for = self.node_id
        self.votes_received = 1  # Vote for self
        return f"Node {self.node_id} starting election for term {self.term}"
    
    def receive_vote(self):
        """Receive a vote from another node"""
        self.votes_received += 1
        if self.votes_received > self.total_nodes // 2:
            self.state = "LEADER"
            return f"Node {self.node_id} elected LEADER with {self.votes_received} votes"
        return None
    
    def __repr__(self):
        return f"Node({self.node_id}, state={self.state}, term={self.term})"

# Simulate election
print("Raft Leader Election Simulation")
print("="*50)

nodes = [Node(i, 5) for i in range(5)]

# Simulate timeout on node 2
candidate = nodes[2]
print(candidate.start_election())

# Other nodes vote
for i in [0, 1, 4]:  # 3 votes + self = 4 votes (majority)
    result = candidate.receive_vote()
    if result:
        print(result)
        break

print(f"\nFinal states: {nodes}")

Raft Leader Election Simulation
Node 2 starting election for term 1
Node 2 elected LEADER with 3 votes

Final states: [Node(0, state=FOLLOWER, term=0), Node(1, state=FOLLOWER, term=0), Node(2, state=LEADER, term=1), Node(3, state=FOLLOWER, term=0), Node(4, state=FOLLOWER, term=0)]


---

## Part 5: Reliability & Fault Tolerance

### 5.1 Failure Modes

| Failure Type | Detection | Mitigation |
|--------------|-----------|------------|
| Network partition | Heartbeat timeout | Primary/secondary failover |
| Process crash | Health checks | Automatic restart |
| Data corruption | Checksums | WAL, replicas |
| Slow degradation | Latency monitoring | Circuit breaker |

### 5.2 Circuit Breaker Pattern

```
         ┌──────────────┐
         │    CLOSED    │  Normal operation
         └──────┬───────┘
                │ Failures > threshold
                ▼
         ┌──────────────┐
         │     OPEN     │  Fast fail, no requests
         └──────┬───────┘
                │ Timeout expires
                ▼
         ┌──────────────┐
         │  HALF-OPEN   │  Allow test request
         └──────┬───────┘
                │
       Success? │ Failure?
          ▼     │     ▼
       CLOSED   │   OPEN
```

In [7]:
# Circuit Breaker Implementation

from enum import Enum

class CircuitState(Enum):
    CLOSED = "CLOSED"
    OPEN = "OPEN"
    HALF_OPEN = "HALF_OPEN"

class CircuitBreaker:
    """
    Circuit breaker for protecting against cascading failures.
    Essential for trading system resilience.
    """
    
    def __init__(self, failure_threshold: int = 5, 
                 recovery_timeout: float = 30.0,
                 success_threshold: int = 3):
        self.failure_threshold = failure_threshold
        self.recovery_timeout = recovery_timeout
        self.success_threshold = success_threshold
        
        self.state = CircuitState.CLOSED
        self.failure_count = 0
        self.success_count = 0
        self.last_failure_time = 0
    
    def call(self, func, *args, **kwargs):
        """Execute function through circuit breaker"""
        if not self._can_execute():
            raise Exception("Circuit OPEN - request rejected")
        
        try:
            result = func(*args, **kwargs)
            self._on_success()
            return result
        except Exception as e:
            self._on_failure()
            raise
    
    def _can_execute(self) -> bool:
        if self.state == CircuitState.CLOSED:
            return True
        
        if self.state == CircuitState.OPEN:
            if time.time() - self.last_failure_time >= self.recovery_timeout:
                self.state = CircuitState.HALF_OPEN
                return True
            return False
        
        return True  # HALF_OPEN allows test requests
    
    def _on_success(self):
        if self.state == CircuitState.HALF_OPEN:
            self.success_count += 1
            if self.success_count >= self.success_threshold:
                self.state = CircuitState.CLOSED
                self.failure_count = 0
                self.success_count = 0
    
    def _on_failure(self):
        self.failure_count += 1
        self.last_failure_time = time.time()
        
        if self.state == CircuitState.HALF_OPEN:
            self.state = CircuitState.OPEN
        elif self.failure_count >= self.failure_threshold:
            self.state = CircuitState.OPEN

# Demo
cb = CircuitBreaker(failure_threshold=3, recovery_timeout=1.0)

def unreliable_service():
    if random.random() < 0.5:
        raise Exception("Service unavailable")
    return "Success!"

print("Circuit Breaker Demo")
print("="*40)

for i in range(10):
    try:
        result = cb.call(unreliable_service)
        print(f"Call {i+1}: {result} [State: {cb.state.value}]")
    except Exception as e:
        print(f"Call {i+1}: {e} [State: {cb.state.value}]")

Circuit Breaker Demo
Call 1: Success! [State: CLOSED]
Call 2: Success! [State: CLOSED]
Call 3: Success! [State: CLOSED]
Call 4: Service unavailable [State: CLOSED]
Call 5: Service unavailable [State: CLOSED]
Call 6: Success! [State: CLOSED]
Call 7: Service unavailable [State: OPEN]
Call 8: Circuit OPEN - request rejected [State: OPEN]
Call 9: Circuit OPEN - request rejected [State: OPEN]
Call 10: Circuit OPEN - request rejected [State: OPEN]


---

## Part 6: Production Monitoring

### 6.1 Key Metrics for Trading Systems

| Metric Category | Examples | Alert Threshold |
|-----------------|----------|------------------|
| **Latency** | P50, P99, P999 | P99 > 10ms |
| **Throughput** | Messages/sec, Orders/sec | Drop > 20% |
| **Error Rate** | Failed orders, Rejections | > 1% |
| **Resource** | CPU, Memory, Network | > 80% |
| **Business** | P&L, Position, Risk | Breach limits |

### 6.2 The Four Golden Signals (Google SRE)

1. **Latency**: Time to service a request
2. **Traffic**: Demand on the system
3. **Errors**: Rate of failed requests
4. **Saturation**: System utilization

### 6.3 Observability Stack

```
┌─────────────────────────────────────────────────────────────┐
│                    OBSERVABILITY STACK                       │
├─────────────────────────────────────────────────────────────┤
│                                                              │
│  METRICS          LOGS              TRACES                   │
│  ────────         ────              ──────                   │
│  Prometheus       ELK Stack         Jaeger                   │
│  Grafana          Splunk            Zipkin                   │
│  InfluxDB         CloudWatch        OpenTelemetry            │
│                                                              │
│              ┌─────────────────────┐                         │
│              │    ALERTING         │                         │
│              │  PagerDuty/OpsGenie │                         │
│              └─────────────────────┘                         │
│                                                              │
└─────────────────────────────────────────────────────────────┘
```

In [8]:
# Metrics Collection System

from collections import defaultdict
import statistics

class MetricsCollector:
    """
    Simple metrics collector for trading systems.
    In production, use Prometheus/StatsD.
    """
    
    def __init__(self):
        self.counters: Dict[str, int] = defaultdict(int)
        self.gauges: Dict[str, float] = {}
        self.histograms: Dict[str, List[float]] = defaultdict(list)
    
    def increment(self, name: str, value: int = 1):
        """Increment a counter"""
        self.counters[name] += value
    
    def set_gauge(self, name: str, value: float):
        """Set a gauge value"""
        self.gauges[name] = value
    
    def observe(self, name: str, value: float):
        """Record histogram observation"""
        self.histograms[name].append(value)
    
    def get_percentile(self, name: str, percentile: float) -> float:
        """Get percentile from histogram"""
        if name not in self.histograms:
            return 0.0
        return np.percentile(self.histograms[name], percentile)
    
    def report(self):
        """Generate metrics report"""
        print("\n📊 METRICS REPORT")
        print("="*50)
        
        print("\nCounters:")
        for name, value in self.counters.items():
            print(f"  {name}: {value}")
        
        print("\nGauges:")
        for name, value in self.gauges.items():
            print(f"  {name}: {value:.2f}")
        
        print("\nHistograms (Latency):")
        for name, values in self.histograms.items():
            if values:
                print(f"  {name}:")
                print(f"    P50: {np.percentile(values, 50):.2f}μs")
                print(f"    P99: {np.percentile(values, 99):.2f}μs")
                print(f"    P999: {np.percentile(values, 99.9):.2f}μs")

# Demo
metrics = MetricsCollector()

# Simulate trading activity
for _ in range(1000):
    # Record latencies
    latency = np.random.exponential(100)  # μs
    metrics.observe('order_latency', latency)
    
    # Count events
    metrics.increment('orders_processed')
    if random.random() < 0.02:  # 2% error rate
        metrics.increment('order_errors')

# Set current state
metrics.set_gauge('position_aapl', 1500)
metrics.set_gauge('unrealized_pnl', 25000.50)

metrics.report()


📊 METRICS REPORT

Counters:
  orders_processed: 1000
  order_errors: 16

Gauges:
  position_aapl: 1500.00
  unrealized_pnl: 25000.50

Histograms (Latency):
  order_latency:
    P50: 69.86μs
    P99: 440.20μs
    P999: 618.42μs


---

## Interview Questions

### Conceptual
1. What's the difference between event-driven and request-response architecture?
2. How would you handle a market data feed failure?
3. Explain CAP theorem in the context of order management.

### Technical
1. How do you achieve sub-millisecond latency?
2. Design a lock-free order book.
3. How would you implement exactly-once processing?

### Design
1. Design a market data distribution system.
2. Design an order management system for multi-asset trading.
3. How would you handle 10M messages per second?

---

## Key Takeaways

| Concept | Key Points |
|---------|------------|
| Architecture | Event-driven for flexibility, shared-memory for speed |
| Latency | Minimize allocations, use binary protocols |
| Data | Hot/warm/cold tiers, time-series DBs |
| Distributed | CAP trade-offs, consensus for coordination |
| Reliability | Circuit breakers, failover, monitoring |

---

## References

- *Designing Data-Intensive Applications* - Martin Kleppmann
- *Building Trading Systems* - Harris
- *Site Reliability Engineering* - Google
- *LMAX Architecture* - Martin Fowler